In [5]:
import pandas as pd

nombres_jornadas = {
    "J1": "Apertura - J1 - Local vs Universidad Cesar Vallejo",
    "J2": "Apertura - J2 - Visita vs Alianza Atlético de Sullana",
    "J3": "Apertura - J3 - Local vs Universitario de Deportes",
    "J4": "Apertura - J4 - Visita vs Unión Comercio",
    "J5": "Apertura - J5 - Local vs Comerciantes Unidos",
    "J6": "Apertura - J6 - Visita vs ADT",
    "J7": "Apertura - J7 - Local vs Sporting Cristal",
    "J8": "Apertura - J8 - Visita vs Cienciano",
    "J9": "Apertura - J9 - Local vs Los Chankas",
    #"J10": "Apertura - J10 - Visita vs Carlos Manucci",
    "C1": "Copa Libertadores - J1 - Local vs Fluminense", 
    #"C2": "Copa Libertadores - J2 - Visita vs Cerro Porteño"
}

jugadores_df_paths = [
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\J1_AL_stats_jugadores.csv',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\J2_AL_stats_jugadores.csv',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\J3_AL_stats_jugadores.csv',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\J4_AL_stats_jugadores.csv',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\J5_AL_stats_jugadores.csv',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\J6_AL_stats_jugadores.csv',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\J7_AL_stats_jugadores.csv',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\J8_AL_stats_jugadores.csv',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\J9_AL_stats_jugadores.csv',
    r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\C1_AL_stats_jugadores.csv',
]



# Lista para almacenar los DataFrames de cada jornada
dfs = []

# Leer cada archivo y agregarlo a la lista de DataFrames
for path, jornada in zip(jugadores_df_paths, nombres_jornadas):
    df_temp = pd.read_csv(path)
    df_temp['Jornada'] = nombres_jornadas[jornada]  # Añade la columna 'Jornada'
    dfs.append(df_temp)

# Combinar todos los DataFrames en uno solo
df_combinado = pd.concat(dfs, ignore_index=True)

# Exportar a Excel
df_combinado.to_excel(r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\Resumen_AL_Jugadores.xlsx', index=False)


In [7]:
import pandas as pd

# Rutas a los archivos
resumen_jugadores_path = r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\Resumen_AL_Jugadores.xlsx'
jugadores_2024_path = r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\ALIANZA LIMA 2024.xlsx'
a_jugadores_2024_path = r'C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStats\Archivos para el tablero final\ALIANZA LIMA 2024 AUTOMATIZADO.xlsx'

# Carga de los DataFrames
df_resumen = pd.read_excel(resumen_jugadores_path)
df_jugadores = pd.read_excel(jugadores_2024_path)

# Inicializar a 0 las columnas de acumulados antes de empezar el procesamiento
df_jugadores['Minutos'] = 0
df_jugadores['Goles'] = 0
df_jugadores['Asistencias'] = 0

# Acumular Goles y Asistencias de forma global
goles_asistencias_acumulados = df_resumen.groupby('name')[['goalAssist','goals']].sum().reset_index()

# Actualizar df_jugadores con los valores acumulados de Goles y Asistencias
for _, row in goles_asistencias_acumulados.iterrows():
    nombre_jugador = row['name']
    goles = row['goals']
    asistencias = row['goalAssist']
    idx = df_jugadores.index[df_jugadores['Jugador'] == nombre_jugador].tolist()
    if idx:
        df_jugadores.at[idx[0], 'Goles'] = goles
        df_jugadores.at[idx[0], 'Asistencias'] = asistencias


# Mapeo de jornadas detallado a corto
mapeo_jornadas = {
    "Apertura - J1 - Local vs Universidad Cesar Vallejo" : "J1",
    "Apertura - J2 - Visita vs Alianza Atlético de Sullana" : "J2",
    "Apertura - J3 - Local vs Universitario de Deportes" : "J3",
    "Apertura - J4 - Visita vs Unión Comercio" : "J4",
    "Apertura - J5 - Local vs Comerciantes Unidos" : "J5",
    "Apertura - J6 - Visita vs ADT": "J6",
    "Apertura - J7 - Local vs Sporting Cristal": "J7",
    "Apertura - J8 - Visita vs Cienciano": "J8",
    "Apertura - J9 - Local vs Los Chankas": "J9",
    #"Apertura - J10 - Visita vs Carlos Manucci": "J10" ,
    "Copa Libertadores - J1 - Local vs Fluminense" : "C1", 
    #"C2": "Copa Libertadores - J2 - Visita vs Cerro Porteño"
}


# Procesamiento y actualización de df_jugadores con datos de df_resumen
for jornada_detalle, jornada_corta in mapeo_jornadas.items():
    filtro_jornada = df_resumen['Jornada'].str.contains(jornada_detalle, na=False)
    df_filtrado = df_resumen[filtro_jornada]

    for _, row in df_filtrado.iterrows():
        nombre_jugador = row['name']
        idx = df_jugadores.index[df_jugadores['Jugador'] == nombre_jugador].tolist()
        if idx:
            idx = idx[0]
            for col in ['Minutos Jugados']:
                valor = pd.to_numeric(row.get(col, 0), errors='coerce')
                if col == 'Minutos Jugados':
                    df_jugadores.at[idx, 'Minutos'] += valor
                    df_jugadores.at[idx, f'{jornada_corta} - Minutos'] = valor


# Guardar el DataFrame actualizado
df_jugadores.to_excel(a_jugadores_2024_path, index=False)